# Centralized Model Competition
Instructions:
1. Run the file `Centralized_Model.ipynb` in Google Colab environment
2. Load the given dataset in the following directory `/content/`
3. Create a folder `data` in the following directory `/content/` to store the .csv files for the predictions made by the model for each stations.  
4. After the execution is completed, download `data.zip` which contains the prediction values for all the stations.



In [ ]:
# Import libraries
import pandas as pd
import tensorflow as tf
import missingno as msno
import re
import os
import datetime
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import r2_score

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

OUT_STEPS = 24
MAX_EPOCHS = 20
num_features = 4
n = 33293  # 80% of index

# Helper Functions
# Function to create Dataframe for each station
def create_df(PATH):
    # Loading the data
    df = pd.read_csv(PATH,sep=';',skiprows=12)
    df = df.drop(columns=['Slut'])
    column_indices = {i: name for i, name in enumerate(df.columns)}
    # Renaming column names
    for i in range(0,len(column_indices)):
        column = column_indices[i]
        if column.startswith('Black Carbon'):
            df.rename(columns = lambda x: re.sub('Black Carbon.*','Black Carbon',x), inplace = True)
        if column.startswith('CO'):
            df.rename(columns = lambda x: re.sub('CO.*','CO',x), inplace = True)
        if column.startswith('O3'):
            df.rename(columns = lambda x: re.sub('O3.*','O3',x), inplace = True)
        if column.startswith('NO2'):
            df.rename(columns = lambda x: re.sub('NO2.*','NO2',x), inplace = True)
        if column.startswith('NOX as NO2'):
            df.rename(columns = lambda x: re.sub('NOX as NO2.*','NOX as NO2',x), inplace = True)
        if column.startswith('PM10'):
            df.rename(columns = lambda x: re.sub('PM10.*','PM10',x), inplace = True)
        if column.startswith('PM2.5'):
            df.rename(columns = lambda x: re.sub('PM2.5.*','PM2.5',x), inplace = True)
    
    df = df.set_index('Start')
    df1 = pd.DataFrame()
    # Adding Air Pressure from Stockholm Station
    pressure = pd.read_csv('/content/stockholm-airpressure.csv',sep=';',skiprows= 8)
    pressure = pressure.drop(columns=['Tidsutsnitt:','Unnamed: 4','Kvalitet'])
    pressure.rename(columns = lambda x: re.sub('Lufttryck.*','Air Pressure',x), inplace = True)
    pressure.rename(columns = lambda x: re.sub('Tid.*','Time',x), inplace = True) 
    pressure.rename(columns = lambda x: re.sub('Datum.*','Date',x), inplace = True) 
    pressure['Start']= pd.to_datetime(pressure['Date'] + ' ' + pressure['Time'])
    pressure = pressure.drop(columns=['Date','Time'])
    pressure = pressure[(pressure['Start']>= "2015-01-01 00:00:00")]
    pressure = pressure[(pressure['Start']<="2019-12-31 23:00:00")]
    pressure = pressure.set_index('Start')
    df1 = df1.merge(pressure, left_index=True, right_index=True ,how='outer')
    
    # Adding Air Temperature from Stockholm Station
    temp = pd.read_csv('/content/stockholm-airtemp.csv',sep=';',skiprows= 8)
    temp = temp.drop(columns=['Tidsutsnitt:','Unnamed: 4','Kvalitet']) 
    temp.rename(columns = lambda x: re.sub('Lufttemperatur.*','Air Temperature',x), inplace = True)
    temp.rename(columns = lambda x: re.sub('Tid.*','Time',x), inplace = True) 
    temp.rename(columns = lambda x: re.sub('Datum.*','Date',x), inplace = True) 
    temp['Start']= pd.to_datetime(temp['Date'] + ' ' + temp['Time'])
    temp = temp.drop(columns=['Date','Time'])
    temp = temp[(temp['Start']>="2015-01-01 00:00:00")]
    temp = temp[(temp['Start']<="2019-12-31 23:00:00")]
    temp = temp.set_index('Start')
    df1 = df1.merge(temp, left_index=True, right_index=True ,how='outer')
    
    # Adding Relative Humidity data from Stockholm Station
    humidity = pd.read_csv('/content/stockholm-humidity.csv',sep=';',skiprows= 8)
    humidity = humidity.drop(columns=['Tidsutsnitt:','Unnamed: 4','Kvalitet']) 
    humidity.rename(columns = lambda x: re.sub('Relativ Luftfuktighet.*','Humidity',x), inplace = True)
    humidity.rename(columns = lambda x: re.sub('Tid.*','Time',x), inplace = True) 
    humidity.rename(columns = lambda x: re.sub('Datum.*','Date',x), inplace = True) 
    humidity['Start']= pd.to_datetime(humidity['Date'] + ' ' + humidity['Time'])
    humidity = humidity.drop(columns=['Date','Time'])
    humidity = humidity[(humidity['Start']>= "2015-01-01 00:00:00")]
    humidity = humidity[(humidity['Start']<="2019-12-31 23:00:00")]
    humidity = humidity.set_index('Start')
    df1 = df1.merge(humidity, left_index=True, right_index=True ,how='outer')
    cols = ['Air Pressure','Air Temperature','Humidity']
    df1.loc[:,cols] = df1.loc[:,cols].ffill()
    df1.loc[:,cols] = df1.loc[:,cols].bfill()
    df = df.merge(df1, left_index=True, right_index=True, how='outer')
    df.reset_index(drop=False, inplace= True)
    
    # Converting datatype of feature 'Start' to datetime
    df['Start'] = pd.to_datetime(df['Start'])
    df['Year'] = df['Start'].dt.year
    df['Month'] = df['Start'].dt.month
    df['Week'] = df['Start'].dt.week
    df['DayOfWeek'] = df['Start'].dt.day_name()
    df = pd.concat([df,pd.get_dummies(df['DayOfWeek'])],axis=1)
    weekend={'Sunday':1,'Monday':0,'Tuesday':0,'Wednesday':0,'Thursday':0,'Friday':0,'Saturday':1}
    weekday={'Sunday':0,'Monday':1,'Tuesday':1,'Wednesday':1,'Thursday':1,'Friday':1,'Saturday':0}
    season={1:'Winter', 2:'Winter', 3:'Spring', 4:'Spring', 5:'Spring', 6:'Summer', 7:'Summer', 8:'Summer', 9:'Fall', 10:'Fall', 11:'Fall', 12:'Winter'} 
    df['Weekend']= df['DayOfWeek'].map(weekend)
    df['Weekday']= df['DayOfWeek'].map(weekday)
    df['Season']= df['Month'].map(season)
    df = pd.concat([df,pd.get_dummies(df['Season'])],axis=1)
    df = df.drop(columns=['DayOfWeek','Year','Month','Week','Season','Air Temperature'])
    return df

# Function to display the Missing Values
def missingstats():  
  print('Station 1 Missing Values Stats:')
  print('----------------------------------------------')
  print('\nNo of Zeros Entries:\n',(station1.select_dtypes(include=['float64']) == 0).astype(int).sum(axis=0))
  print('\nNo of Negative Entries:\n',(station1.select_dtypes(include=['float64']) < 0).astype(int).sum(axis=0))
  print('\nNo of Null Entries:\n',(station1.select_dtypes(include=['float64']).isnull()).astype(int).sum(axis=0))
  print('\n\n')

  print('Station 2 Missing Values Stats:')
  print('----------------------------------------------')
  print('\nNo of Zeros Entries:\n',(station2.select_dtypes(include=['float64']) == 0).astype(int).sum(axis=0))
  print('\nNo of Negative Entries:\n',(station2.select_dtypes(include=['float64']) < 0).astype(int).sum(axis=0))
  print('\nNo of Null Entries:\n',(station2.select_dtypes(include=['float64']).isnull()).astype(int).sum(axis=0))
  print('\n\n')

  print('Station 3 Missing Values Stats:')
  print('----------------------------------------------')
  print('\nNo of Zeros Entries:\n',(station3.select_dtypes(include=['float64']) == 0).astype(int).sum(axis=0))
  print('\nNo of Negative Entries:\n',(station3.select_dtypes(include=['float64']) < 0).astype(int).sum(axis=0))
  print('\nNo of Null Entries:\n',(station3.select_dtypes(include=['float64']).isnull()).astype(int).sum(axis=0))
  print('\n\n')

  print('Station 4 Missing Values Stats:')
  print('----------------------------------------------')
  print('\nNo of Zeros Entries:\n',(station4.select_dtypes(include=['float64']) == 0).astype(int).sum(axis=0))
  print('\nNo of Negative Entries:\n',(station4.select_dtypes(include=['float64']) < 0).astype(int).sum(axis=0))
  print('\nNo of Null Entries:\n',(station4.select_dtypes(include=['float64']).isnull()).astype(int).sum(axis=0))
  print('\n\n')
  return None


# Function to calculate SMAPE
def smape(actual, predicted):
   dividend= np.abs(np.array(actual) - np.array(predicted))
   denominator = np.array(actual) + np.array(predicted)
   return 2 * np.mean(np.divide(dividend, denominator, out=np.zeros_like(dividend), where=denominator!=0, casting='unsafe'))

PATH1 = '/content/shair-8779-1-6-3.csv'
PATH2 = '/content/shair-8780-1-6-3.csv'
PATH3 = '/content/shair-8781-1-6-1.csv'
PATH4 = '/content/shair-18644-1-6-3.csv'
station1 = create_df(PATH1)
station2 = create_df(PATH2)
station3 = create_df(PATH3)
station4 = create_df(PATH4)
smape_values = pd.DataFrame(columns=['Station','NO2','NOX as NO2','PM10','PM2.5','Average'])
r2_scores = pd.DataFrame(columns=['Station','NO2','NOX as NO2','PM10','PM2.5'])


station1 = station1.set_index('Start')
station1[station1 < 0] = 0
station1.reset_index(drop=False, inplace= True)
station1 = station1.interpolate(method ='linear', limit_direction ='forward')

station2 = station2.set_index('Start')
station2[station2 < 0] = 0
station2.reset_index(drop=False, inplace= True)
station2 = station2.interpolate(method ='linear', limit_direction ='forward')

station3 = station3.set_index('Start')
station3[station3 < 0] = 0
station3.reset_index(drop=False, inplace= True)
station3 = station3.interpolate(method ='linear', limit_direction ='forward')

station4 = station4.set_index('Start')
station4[station4 < 0] = 0
station4.reset_index(drop=False, inplace= True)
station4 = station4.interpolate(method ='linear', limit_direction ='forward')
#missingstats()

for count in range(1,5):
  print('\nStation-',count)
  print('----------------------------------------------')
  if count == 1:
  #Split for station 1
    index = station1[station1['Start']=='2019-09-30 00:00:00'].index.values.astype(int)[0]
    date_time = pd.to_datetime(station1.pop('Start'), format='%d.%m.%Y %H:%M:%S')
    column_indices = {name: i for i, name in enumerate(station1.columns)}
    train_df = station1.iloc[:n,:]
    val_df = station1.iloc[n:index,:]
    test_df = station1.iloc[index:,:]
    station = 8779

  if count == 2:
  #Split for station 2
    index = station2[station2['Start']=='2019-09-30 00:00:00'].index.values.astype(int)[0]
    date_time = pd.to_datetime(station2.pop('Start'), format='%d.%m.%Y %H:%M:%S')
    column_indices = {name: i for i, name in enumerate(station2.columns)}
    train_df = station2.iloc[:n,:]
    val_df = station2.iloc[n:index,:]
    test_df = station2.iloc[index:,:]
    station = 8780

  if count == 3:
    #Split for station 3
    index = station3[station3['Start']=='2019-09-30 00:00:00'].index.values.astype(int)[0]
    date_time = pd.to_datetime(station3.pop('Start'), format='%d.%m.%Y %H:%M:%S')
    column_indices = {name: i for i, name in enumerate(station3.columns)}
    train_df = station3.iloc[:n,:]
    val_df = station3.iloc[n:index,:]
    test_df = station3.iloc[index:,:]
    station = 8781

  #Split for station 4
  if count == 4:
    index = station4[station4['Start']=='2019-09-30 00:00:00'].index.values.astype(int)[0]
    date_time = pd.to_datetime(station4.pop('Start'), format='%d.%m.%Y %H:%M:%S')
    column_indices = {name: i for i, name in enumerate(station4.columns)}
    train_df = station4.iloc[:n,:]
    val_df = station4.iloc[n:index,:]
    test_df = station4.iloc[index:,:]
    station = 18644
  
  # Normalization
  train_mean = train_df.mean()
  train_std = train_df.std()
  train_df = (train_df - train_mean) / train_std
  val_df = (val_df - train_mean) / train_std
  test_df = (test_df - train_mean) / train_std
  if count == 1:
    df_std = (station1 - train_mean) / train_std

  if count == 2:
    df_std = (station2 - train_mean) / train_std
  
  if count == 3:
    df_std = (station3 - train_mean) / train_std
  
  if count == 4:
    df_std = (station4 - train_mean) / train_std

  #df_std = df_std.melt(var_name='Column', value_name='Normalized')
  #plt.figure(figsize=(12, 6))
  #ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
  #_ = ax.set_xticklabels(station4.keys(), rotation=90)

  # Class to define a window
  class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
                train_df=train_df, val_df=val_df, test_df=test_df,
                label_columns=None):

      self.train_df = train_df
      self.val_df = val_df
      self.test_df = test_df

      self.label_columns = label_columns
      if label_columns is not None:
        self.label_columns_indices = {name: i for i, name in
                                      enumerate(label_columns)}
      self.column_indices = {name: i for i, name in
                            enumerate(train_df.columns)}

      self.input_width = input_width
      self.label_width = label_width
      self.shift = shift

      self.total_window_size = input_width + shift

      self.input_slice = slice(0, input_width)
      self.input_indices = np.arange(self.total_window_size)[self.input_slice]

      self.label_start = self.total_window_size - self.label_width
      self.labels_slice = slice(self.label_start, None)
      self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
      return '\n'.join([
          f'Total window size: {self.total_window_size}',
          f'Input indices: {self.input_indices}',
          f'Label indices: {self.label_indices}',
          f'Label column name(s): {self.label_columns}'])
  # Function to Split window
  def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
      labels = tf.stack(
          [labels[:, :, self.column_indices[name]] for name in self.label_columns],
          axis=-1)
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels

  # Function to Plot
  def plot(self, model=None, plot_col='PM10', max_subplots=3):
    inputs, labels = self.example
    plt.figure(figsize=(12, 8))
    plot_col_index = self.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))
    for n in range(max_n):
      plt.subplot(3, 1, n+1)
      plt.ylabel(f'{plot_col} [normed]')
      plt.plot(self.input_indices, inputs[n, :, plot_col_index],
              label='Inputs', marker='.', zorder=-10)

      if self.label_columns:
        label_col_index = self.label_columns_indices.get(plot_col, None)
      else:
        label_col_index = plot_col_index

      if label_col_index is None:
        continue

      plt.scatter(self.label_indices, labels[n, :, label_col_index],
                  edgecolors='k', label='Labels', c='#2ca02c', s=64)
      if model is not None:
        predictions = model(inputs)
        plt.scatter(self.label_indices, predictions[n, :, label_col_index], 
                    marker='X', edgecolors='k', label='Predictions',
                    c='#ff7f0e', s=64)
      if n == 0:
        plt.legend()
    plt.xlabel('Time [h]')

  # Function to Make Dataset
  def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,)

    ds = ds.map(self.split_window)

    return ds

  @property
  def train(self):
    return self.make_dataset(self.train_df)

  @property
  def val(self):
    return self.make_dataset(self.val_df)

  @property
  def test(self):
    return self.make_dataset(self.test_df)

  @property
  def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
      result = next(iter(self.train))
      self._example = result
    return result

  # Function to Compile the model 
  def compile_and_fit(model, window, patience=2):
      
      early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        patience=patience,
                                                        mode='min')
      def smape(y_true, y_pred):
            y_true = y_true * train_std[:num_features] + train_mean[:num_features]
            y_pred = y_pred * train_std[:num_features] + train_mean[:num_features]
            return tf.reduce_mean(2 * tf.abs(y_true - y_pred)
                                  / (tf.abs(y_pred) + tf.abs(y_true)), axis=-1)
      model.compile(loss=tf.losses.MeanSquaredError(),
                    optimizer=tf.optimizers.Adam(),
                    metrics=[smape,tf.metrics.MeanAbsoluteError()])
      history = model.fit(window.train, epochs=MAX_EPOCHS,
                        validation_data=window.val,
                        callbacks=[early_stopping])
      return history

  lstm_model = tf.keras.Sequential([
      tf.keras.layers.LSTM(32, return_sequences=False),
      tf.keras.layers.Dense(OUT_STEPS*num_features,
                            kernel_initializer=tf.initializers.zeros),
      tf.keras.layers.Reshape([OUT_STEPS, num_features])
    ])

  WindowGenerator.split_window = split_window
  WindowGenerator.plot = plot
  WindowGenerator.make_dataset = make_dataset
  WindowGenerator.train = train
  WindowGenerator.val = val
  WindowGenerator.test = test
  WindowGenerator.example = example
  
  multi_window = WindowGenerator(input_width=24,
                                label_width=OUT_STEPS, 
                                shift=24,label_columns=['NO2','NOX as NO2','PM10','PM2.5'])

  history = compile_and_fit(lstm_model,multi_window)
  multi_val_performance = {}
  multi_performance = {}
  multi_val_performance['LSTM'] = lstm_model.evaluate(multi_window.val)
  multi_performance['LSTM'] = lstm_model.evaluate(multi_window.test, verbose=1)
  #multi_window.plot(multi_lstm_model)

  predict = pd.DataFrame(columns=['NO2','NOX as NO2','PM10','PM2.5'])
  idx= 0
  for j in range(0,92):
    test = test_df.iloc[idx:idx+48,:]
    predict_window = WindowGenerator(input_width=24,
                                label_width=24, 
                                test_df = test,
                                shift=24,label_columns=['NO2','NOX as NO2','PM10','PM2.5'])
    values = lstm_model.predict(predict_window.test)
    for i in range(0,24):
        dictionary = {'NO2':values[0][i][0], 'NOX as NO2':values[0][i][1],'PM10':values[0][i][2], 'PM2.5':values[0][i][3]}
        predict = predict.append(dictionary,ignore_index=True)
    idx=idx+24
  predict= (predict*train_std)+train_mean
  predict.drop(columns=['Air Pressure', 'Fall', 'Friday', 'Humidity', 'Monday',
          'Saturday', 'Spring', 'Summer', 'Sunday',
          'Thursday', 'Tuesday', 'Wednesday', 'Weekday', 'Weekend', 'Winter'])
  predict['Start'] = pd.date_range(start='2019-10-01 00:00:00', periods=len(predict), freq='H')
  predict = predict[['Start','NO2','NOX as NO2','PM10','PM2.5']]
  predict.to_csv(r'/content/data/'+str(station)+'.csv', index = False)
  
  # Evaluating Actual vs Predicted values
  if count == 1:
    actual=station1.iloc[41616:,:]
  
  if count == 2:
    actual=station2.iloc[41616:,:]

  if count == 3:
    actual=station3.iloc[41616:,:]

  if count == 4:
    actual=station4.iloc[41616:,:]

  actual.index = range(len(predict))
  no2 = smape(actual['NO2'],predict['NO2'])
  no2_r2 = r2_score(actual['NO2'], predict['NO2'])
  nox = smape(actual['NOX as NO2'],predict['NOX as NO2'])
  nox_r2 = r2_score(actual['NOX as NO2'],predict['NOX as NO2'])
  pm2_5 = smape(actual['PM2.5'],predict['PM2.5'])
  pm2_5_r2 = r2_score(actual['PM2.5'], predict['PM2.5'])
  pm10 = smape(actual['PM10'], predict['PM10'])
  pm10_r2 = r2_score(actual['PM10'],predict['PM10']) 
  dict1 = {'Station':str(station),'NO2':no2,'NOX as NO2':nox,'PM2.5':pm2_5,'PM10':pm10, 'Average':(no2+nox+pm2_5+pm10)/4 }
  dict2 = {'Station':str(station),'NO2':no2_r2,'NOX as NO2':nox_r2,'PM2.5':pm2_5_r2,'PM10':pm10_r2}
  smape_values = smape_values.append(dict1,ignore_index=True)
  r2_scores = r2_scores.append(dict2,ignore_index=True)

print('\nSmape Values for each station:')
print(smape_values)
print('\nR2 Scores:')
print(r2_scores)
!zip -r /content/data.zip /content/data

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



Station- 1
----------------------------------------------
Epoch 1/20
1039/1039 [==============================] - 15s 14ms/step - loss: 0.7138 - smape: 0.5464 - mean_absolute_error: 0.5785 - val_loss: 0.6366 - val_smape: 0.5189 - val_mean_absolute_error: 0.5368
Epoch 2/20
1039/1039 [==============================] - 14s 14ms/step - loss: 0.6037 - smape: 0.5109 - mean_absolute_error: 0.5232 - val_loss: 0.6345 - val_smape: 0.5186 - val_mean_absolute_error: 0.5352
Epoch 3/20
1039/1039 [==============================] - 14s 14ms/step - loss: 0.5725 - smape: 0.5035 - mean_absolute_error: 0.5088 - val_loss: 0.6440 - val_smape: 0.5213 - val_mean_absolute_error: 0.5397
Epoch 4/20
69/69 [==============================] - 0s 5ms/step - loss: 0.5044 - smape: 0.5508 - mean_absolute_error: 0.5089

Station- 2
----------------------------------------------
Epoch 1/20
1039/1039 [==============================] - 15s 14ms/step - loss: 0.6884 - smape: 0.4875 - mean_absolute_error: 0.5773 - val_loss: 0.